# Predicting Credit Card Approvals

<p> Commercial banks receive <em>a lot</em> of applications for credit cards, the proccess to manually analyse these applications can be time consuming and error-prone. In this workbook an automatic credit card approval predictor will be built using machine learning techniques </p>
<br>
<p> The dataset used is the <a href="http://archive.ics.uci.edu/ml/datasets/credit+approval">Credit Card Approval dataset</a> from the UCI Machine Learning Repository. </p>

The Structure of this workbook is as follows:
<li> Loading and Viewing the Data </li>
<li> Cleaning the Data </li>
<li> Preprocessing the Data to ensure the machine learning model can make accurate predictions </li>
<li> Fitting a logistic regression Model to the training dataset </li>
<li> Making predictions and evaluating the performance </li>
<li> Grid Searching and improving performance of the model </li>

### 1. Loading and Viewing The Data

In [18]:
# Import pandas
import pandas as pd
# Import Numpy
import numpy as np

# Load dataset
cc_apps = pd.read_csv("Predicting Credit Card Approvals Data.csv", header=None)

# Inspect data
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43,560,+
2,a,24.5,0.500,u,g,q,h,1.50,t,f,0,f,g,280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120,0,+


### 2. Inspection Of The Data

<p> Although the features of this data set have been anonymized for privacy, <a href="http://rstudio-pubs-static.s3.amazonaws.com/73039_9946de135c0a49daa7a0a9eda4a67a72.html">this blog</a> gives us a good overview of the possible features. </p>
    
<p> The possible features in a typical credit card application are <code>Gender</code>, <code>Age</code>, <code>Debt</code>, <code>Married</code>, <code>BankCustomer</code>, <code>EducationLevel</code>, <code>Ethnicity</code>, <code>YearsEmployed</code>, <code>PriorDefault</code>, <code>Employed</code>, <code>CreditScore</code>, <code>DriversLicense</code>, <code>Citizen</code>, <code>ZipCode</code>, <code>Income</code> and <code>ApprovalStatus</code>. </p>

<p> From the inspection of the data we can see that there is a mix of numerical and non-numerical values </p>

In [19]:
# Print summary
cc_apps_description = cc_apps.describe()
print(cc_apps_description)

print("\n")

# Print DataFrame information
print(cc_apps.info())

print("\n")

# Inspect missing values in the dataset
cc_apps.tail(20)

               2           7          10             14
count  690.000000  690.000000  690.00000     690.000000
mean     4.758725    2.223406    2.40000    1017.385507
std      4.978163    3.346513    4.86294    5210.102598
min      0.000000    0.000000    0.00000       0.000000
25%      1.000000    0.165000    0.00000       0.000000
50%      2.750000    1.000000    0.00000       5.000000
75%      7.207500    2.625000    3.00000     395.500000
max     28.000000   28.500000   67.00000  100000.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    object 
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 no

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
670,b,47.17,5.835,u,g,w,v,5.500,f,f,0,f,g,465,150,-
671,b,25.83,12.835,u,g,cc,v,0.500,f,f,0,f,g,0,2,-
672,a,50.25,0.835,u,g,aa,v,0.500,f,f,0,t,g,240,117,-
673,?,29.5,2.000,y,p,e,h,2.000,f,f,0,f,g,256,17,-
674,a,37.33,2.500,u,g,i,h,0.210,f,f,0,f,g,260,246,-
675,a,41.58,1.040,u,g,aa,v,0.665,f,f,0,f,g,240,237,-
676,a,30.58,10.665,u,g,q,h,0.085,f,t,12,t,g,129,3,-
677,b,19.42,7.250,u,g,m,v,0.040,f,t,1,f,g,100,1,-
678,a,17.92,10.210,u,g,ff,ff,0.000,f,f,0,f,g,0,50,-
679,a,20.08,1.250,u,g,c,v,0.000,f,f,0,f,g,0,0,-


### 3.1 Handling the Missing Values

<p> In order for our machine learning model to have the best accuracy the data missing values must be adressed. Values that contain '?' will need to be replaced with NaN </p>

In [20]:
# Inspect missing values in the dataset
print("Number of missing values:", cc_apps.isnull().values.sum())

# Replace the '?'s with NaN
cc_apps = cc_apps.replace("?",np.NaN)

# Inspect the missing values again
cc_apps.tail(20)

Number of missing values: 0


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
670,b,47.17,5.835,u,g,w,v,5.500,f,f,0,f,g,465,150,-
671,b,25.83,12.835,u,g,cc,v,0.500,f,f,0,f,g,0,2,-
672,a,50.25,0.835,u,g,aa,v,0.500,f,f,0,t,g,240,117,-
673,NaN,29.5,2.000,y,p,e,h,2.000,f,f,0,f,g,256,17,-
674,a,37.33,2.500,u,g,i,h,0.210,f,f,0,f,g,260,246,-
675,a,41.58,1.040,u,g,aa,v,0.665,f,f,0,f,g,240,237,-
676,a,30.58,10.665,u,g,q,h,0.085,f,t,12,t,g,129,3,-
677,b,19.42,7.250,u,g,m,v,0.040,f,t,1,f,g,100,1,-
678,a,17.92,10.210,u,g,ff,ff,0.000,f,f,0,f,g,0,50,-
679,a,20.08,1.250,u,g,c,v,0.000,f,f,0,f,g,0,0,-


### 3.2 Handling the Missing Values (Part ii)

<p> Now that all '?' values have been replaced with NaN's, instead of simply removing all NaN values from the data or ignoring them we will use the process of <a href="https://www.sciencedirect.com/topics/mathematics/imputation-method"> Mean Imputation</a> in order to yield better results for this machine learning model</p>




In [21]:
# Impute the missing values with mean imputation
cc_apps.fillna(cc_apps.mean(), inplace=True)

# Count the number of NaNs in the dataset to verify
cc_apps.isnull().values.sum()

67

### 3.3 Handling the missing Values (Part iii)

<p> Now that the missing values have been replaced in the numeric columns, the values in the non numeric columns need to be imputed </p>

In [22]:
# Iterate over each column of cc_apps
for col in cc_apps.columns:
    # Check if the column is of object type
    if cc_apps[col].dtypes == 'object':
        # Impute with the most frequent value
        cc_apps = cc_apps.fillna(cc_apps[col].value_counts().index[0])

# Count the number of NaNs in the dataset and print the counts to verify
cc_apps.isnull().values.sum()

0

### 4.1 Preprocessing the data (part i) 

<p> Now that the all missing values have been handled the next steps are: </p>
<li>Convert the non-numeric data into numeric.</li>
<li>Split the data into train and test sets. </li>
<li>Scale the feature values to a uniform range.</li>

<p> To convert all non-numeric values into numeric we will use a process called <a href="http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html">label encoding</a>.</p>

In [23]:
# Import LabelEncoder
from sklearn.preprocessing import LabelEncoder
# Instantiate LabelEncoder
le = LabelEncoder()
# Iterate over all the values of each column and extract their dtypes
for col in cc_apps.columns:
    # Compare if the dtype is object
    if cc_apps[col].dtypes =='object':
    # Use LabelEncoder to do the numeric transformation
        cc_apps[col]=le.fit_transform(cc_apps[col])

### 4.2 Preprocessing the data (part ii)
<p> Now, we will split our data into train set and test set to prepare our data for two different phases of machine learning modeling: training and testing. </p>

<p>Features like <code>DriversLicense</code> and <code>ZipCode</code> are not as important as the other features in the dataset for predicting credit card approvals. For this reason they will be dropped from the data in order to improve model performance.

In [24]:
# Import train_test_split
from sklearn.model_selection import train_test_split
# Drop the features 11 and 13 and convert the DataFrame to a NumPy array
cc_apps = cc_apps.drop([cc_apps.columns[10], cc_apps.columns[12]], axis=1)
cc_apps = cc_apps.to_numpy()

# Segregate features and labels into separate variables
X,y = cc_apps[:,0:13] , cc_apps[:,13]

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,
                                y,
                                test_size=0.33,
                                random_state=42)

### 4.2 Preprocessing the data (part iii)

<p> The final step in the preprocessing process is scaling the data. We do this because the credit score of a person is their creditworthiness based on their credit history. The higher this number, the more financially trustworthy a person is considered to be. So, a <code>CreditScore</code> of 1 is the highest since we're rescaling all the values to the range of 0-1.</p>

In [25]:
# Import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
# Instantiate MinMaxScaler and use it to rescale X_train and X_test
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX_train = scaler.fit_transform(X_train)
rescaledX_test = scaler.fit_transform(X_test)

### 4.3 Fitting a logistic regression model to the train set

<p> For now we can assume that the features that affect the credit card approval decision process correlated with each other. Because of this correlation, we'll take advantage of the fact that generalized linear models perform well in these cases. </p>

In [26]:
# Import LogisticRegression
from sklearn.linear_model import LogisticRegression
# Instantiate a LogisticRegression classifier with default parameter values
logreg = LogisticRegression()

# Fit logreg to the train set
logreg.fit(rescaledX_train,y_train)

LogisticRegression()

### 5.1 Making predictions and evaluating performance

<p> We will now evaluate our model on the test set with respect to <a href="https://developers.google.com/machine-learning/crash-course/classification/accuracy">classification accuracy</a>. But we will also take a look the model's <a href="http://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/">confusion matrix</a>.

In [27]:
# Import confusion_matrix
from sklearn.metrics import confusion_matrix
# Use logreg to predict instances from the test set and store it
y_pred = logreg.predict(rescaledX_test)

# Get the accuracy score of logreg model and print it
print("Accuracy of logistic regression classifier: ", logreg.score(rescaledX_test, y_test))

# Print the confusion matrix of the logreg model
confusion_matrix(y_test,y_pred)

Accuracy of logistic regression classifier:  0.8377192982456141


array([[93, 10],
       [27, 98]])

### 5.2 Grid searching and making the model perform better

<p> The model has an accuracy of 84% </p>

<p>For the confusion matrix, the first element of the of the first row of the confusion matrix denotes the true negatives meaning the number of negative instances (denied applications) predicted by the model correctly. And the last element of the second row of the confusion matrix denotes the true positives meaning the number of positive instances (approved applications) predicted by the model correctly.</p>

<p> We can perform a <a href="https://machinelearningmastery.com/how-to-tune-algorithm-parameters-with-scikit-learn/">grid search</a> of the model parameters to improve the model's ability to predict credit card approvals.</p>
<p><a href="http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html">scikit-learn's implementation of logistic regression</a> consists of different hyperparameters but we will grid search over the following two:</p>
<ul>
<li>tol</li>
<li>max_iter</li>
</ul>

In [28]:
# Import GridSearchCV
from sklearn.model_selection import GridSearchCV
# Define the grid of values for tol and max_iter
tol = [0.01,0.001,0.0001]
max_iter = [100,150,200]

# Create a dictionary where tol and max_iter are keys and the lists of their values are corresponding values
param_grid = dict(tol=tol, max_iter=max_iter)

### 5.3 Find the best performing model

In [29]:
# Instantiate GridSearchCV with the required parameters
grid_model = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5)

# Use scaler to rescale X and assign it to rescaledX
rescaledX = scaler.fit_transform(X)

# Fit data to grid_model
grid_model_result = grid_model.fit(rescaledX,y)

# Summarize results
best_score, best_params = grid_model_result.best_score_, grid_model_result.best_params_
print("Best: %f using %s" % (best_score, best_params))

Best: 0.852174 using {'max_iter': 100, 'tol': 0.01}
